<a href="https://colab.research.google.com/github/kimjeongsoo20190147/dacon/blob/main/random%2Bxgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
!pip install konlpy
import konlpy
from konlpy.tag import Komoran
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
import subprocess
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm, tqdm_notebook

In [ ]:
# 데이터 로드
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
from sklearn.model_selection import train_test_split

X_train = train_df.drop(['분류'], axis=1) # decided here what we want to use in our model
y_train = train_df["분류"]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=8678686)

X_real_test = test_df
X_real_train = train_df

In [ ]:
# 제목만 분리.

X_train['combined_text'] = X_train['제목']
X_test['combined_text'] = X_test['제목']
# X_real_test
X_train['combined_text'].head(5)

,combined_text
21606,"용인시, 저소득 가정 청소년 120명 장학금 지원"
11374,오후 6시까지 전국서 1천42명 확진 어제보다 19명 적어
19676,"용인시, 민간 마일리지 지역화폐 '와이페이' 전환 사용하는 '시티포인트' 시스템 개발"
46401,"[포토]박민지, 다승을 향한 티샷"
10025,"수원 성남 고양 용인 심야 교통량 많아 토요일↑, 일요일↓"


In [ ]:
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf_both = tfidf_vectorizer.fit_transform(X_train['combined_text'])
# X_test_tfidf_both = tfidf_vectorizer.transform(X_test['combined_text'])

komoran = Komoran()

# 2. 데이터를 형태소 단위로 토큰화 (X_train['combined_text'], X_test['combined_text'] 사용)
tokenized_X_train = [komoran.morphs(text) for text in X_train['combined_text']]
tokenized_X_test = [komoran.morphs(text) for text in X_test['combined_text']]

print("이전: ", tokenized_X_train[:3])

# 토큰화된 제목이랑 키워드랑 합치기
'''
X_train['combined_text'] = X_train['제목'] + ' ' + X_train['키워드']
X_test['combined_text'] = X_test['제목'] + ' ' + X_test['키워드']
X_real_test
X_train['combined_text'].head(5)
'''

X_real_train['combined_text'] = X_real_train['제목'] + ' ' + X_real_train['키워드']
X_real_test['combined_text'] = X_real_test['제목'] + ' ' + X_real_test['키워드']
X_real_train['combined_text'].head(5)

이전:  [['용인시', ',', '저', '소득', '가정', '청소년', '120', '명', '장학금', '지원'], ['오후', '6', '시', '까지', '전국', '서', '1', '천', '42', '명', '확진', '어제', '보다', '19', '명', '적', '어'], ['용인시', ',', '민간', '마일리지', '지역', '화폐', "'", '와이', '페이', "'", '전환', '사용', '하', '는', "'", '시티', '포인트', "'", '시스템', '개발']]


,combined_text
0,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최 용인문화재단,인..."
1,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원 용인,농촌,테마파크,단체,체험객..."
2,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장 용인시,노후,주택,에너지,..."
3,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대 수원,용인,고양시,특례시,..."
4,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할 용인시,스페인,미국,국..."


In [ ]:
stop_words = pd.read_csv("stop_words.csv")
print(stop_words)

remove_stopwords = [[word for word in sen if not word in stop_words ] for sen in tokenized_X_train]
remove_stopwords_test =  [[word for word in sen if not word in stop_words ] for sen in tokenized_X_test]

print("이후:", remove_stopwords[:3], end="")

        아    어찌됏든
0       휴     그위에
1     아이구     게다가
2     아이쿠  점에서 보아
3     아이고  비추어 보아
4       어    고려하면
..    ...     ...
244    인젠      윙윙
245    이젠      오호
246  된바에야      아하
247   된이상     어쨋든
248    만큼   만 못하다

[249 rows x 2 columns]
이후: [['용인시', ',', '저', '소득', '가정', '청소년', '120', '명', '장학금', '지원'], ['오후', '6', '시', '까지', '전국', '서', '1', '천', '42', '명', '확진', '어제', '보다', '19', '명', '적', '어'], ['용인시', ',', '민간', '마일리지', '지역', '화폐', "'", '와이', '페이', "'", '전환', '사용', '하', '는', "'", '시티', '포인트', "'", '시스템', '개발']]

In [ ]:
word2vec_model = Word2Vec(sentences=remove_stopwords, vector_size=230, window=5, min_count=1, workers=4)

def sentence_vector(sentence, model):
    word_vecs = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

X_train_word2vec_both = np.array([sentence_vector(sentence, word2vec_model) for sentence in remove_stopwords])
X_test_word2vec_both = np.array([sentence_vector(sentence, word2vec_model) for sentence in remove_stopwords_test])


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.transform(y_test)

In [55]:
# 데이터 불균형 해결 (SMOTE 사용)
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(sampling_strategy='not majority', random_state=2024)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_word2vec_both, y_train_encoded)

print(Counter(y_train_resampled).items())

# 데이터셋에서 10%만 사용하도록 샘플링 #----------------------#
X_train_sampled, _, y_train_sampled, _ = train_test_split(X_train_resampled, y_train_resampled, test_size=0.7, random_state=2024)

dict_items([(55, 21547), (4, 21547), (20, 21547), (35, 21547), (48, 21547), (33, 21547), (18, 21547), (8, 21547), (7, 21547), (23, 21547), (9, 21547), (11, 21547), (40, 21547), (42, 21547), (38, 21547), (34, 21547), (41, 21547), (36, 21547), (3, 21547), (21, 21547), (10, 21547), (16, 21547), (31, 21547), (19, 21547), (39, 21547), (2, 21547), (29, 21547), (28, 21547), (37, 21547), (14, 21547), (30, 21547), (15, 21547), (6, 21547), (27, 21547), (25, 21547), (50, 21547), (22, 21547), (45, 21547), (43, 21547), (47, 21547), (26, 21547), (1, 21547), (12, 21547), (46, 21547), (0, 21547), (17, 21547), (5, 21547), (53, 21547), (32, 21547), (54, 21547), (24, 21547), (51, 21547), (44, 21547), (49, 21547), (52, 21547), (13, 21547)])


In [56]:
# 랜덤 포레스트 모델 학습
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=75, random_state=2024)
xgb_model = xgb.XGBClassifier(n_estimators=75, learning_rate=0.1, max_depth=6, random_state=2024)

estimators = [('rf', rf_model), ('xgb', xgb_model)]
stacking_model = StackingClassifier(estimators=estimators, final_estimator=xgb.XGBClassifier(random_state=2024))


stacking_model.fit(X_train_sampled, y_train_sampled)




StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=75,
                                                       random_state=2024)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=2024, ...))

In [57]:
# 예측 수행
y_pred_stacking = stacking_model.predict(X_test_word2vec_both)

In [58]:
# F1 스코어 계산 및 출력
f1_stacking = f1_score(y_test_encoded, y_pred_stacking, average='weighted')
print(f"F1 Score: {f1_stacking}")

F1 Score: 0.5406317386647039


In [ ]:
#!pip install scikit-learn

In [ ]:
# 2. 예측 수행
# y_test_pred = rf_model.predict(X_test_word2vec_both)

# 3. 결과 저장: ID와 예측된 분류 결과를 DataFrame으로 변환
test_results = pd.DataFrame({
    'ID': X_test["ID"],    # 테스트 데이터의 ID
    '분류': label_encoder.inverse_transform(y_pred)     # 예측된 분류 값
})

# 4. 결과를 CSV 파일로 저장
test_results.to_csv('submission.csv', index=False)


# CSV 결과 예시
print(test_results.head())